In [35]:
from sklearn import preprocessing as pp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from music21 import converter, instrument, note, chord, stream
import glob
import pickle
from keras.utils import np_utils
from keras.models import Sequential, Model
from keras.layers import Dense, Reshape, Dropout, LSTM, Activation, Input, Lambda, Flatten, Bidirectional
from keras.layers import BatchNormalization as BatchNorm
from keras.callbacks import ModelCheckpoint, History
from keras.models import load_model
from numpy.random import multinomial as randm
from keras.layers.advanced_activations import LeakyReLU

In [50]:
sequence_length_global = 32

In [51]:
# Current Counts = Current Counts + Note Length % 16 ?

def buildNotes():
    
    #print("In buildNotes()")
    
    notes = []
    noteLengths = []
    
    for file in glob.glob("Music/*.mid"):
        
        try:
            midi = converter.parse(file)    
            
        except:
            print("MIDI file %s failed to parse" % file)
            continue
            
        print("Parsing %s" % file)
        notesToParse = None
        
        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notesToParse = s2.parts[0].recurse()
            print("Instrument Parts %d" % len(s2.parts))
            
        except: # file has notes in a flat structure
            notesToParse = midi.flat.notes
            
        # For each note in notesToParse (a stream of notes)
            
        for element in notesToParse:
            
            # Interesting parameters of notes that we may want to look at here.
            
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
                noteLengths.append(str(float(element.quarterLength)))
                
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
                noteLengths.append(str(float(element.quarterLength)))

    pickle.dump(notes, open('notes.p', 'wb'))
    pickle.dump(noteLengths, open('noteLengths.p', 'wb'))

    return notes, noteLengths

def prepareSeq(notes, noteLengths):

    seqLength = sequence_length_global # S Sequence Length
    inputSize = len(notes) - seqLength # N Samples
    categories = 2 # C Categories
    
    # Yields N x S x C array
    
    pitchSet = sorted(set(notes))
    npitch = len(pitchSet)
    noteToInt = dict((note, number) for number, note in enumerate(pitchSet))
    
    lengthSet = sorted(set(noteLengths))
    nlengths = len(lengthSet)
    lengthToInt = dict((length, number) for number, length in enumerate(lengthSet))
    
    # We only have 2 dimensions at the moment, pitch and length
    # noteIn - a collection of categorical arrays, each row is one category
    # noteOut - the integer which converts to the index of that category's respective
    # one hot encoding.  These categorical one hot encodings are later concatenated
    # columnwise to form a multi hot encoding.

    # This representation is a C x N x S array
    
    noteIn = [
        [],
        []
    ]
    
    noteOut = [
        [],
        []
    ]

    # Mapping of i:i+seqLength notes to note i + seqLength
    
    # X[0] sequence_in : [seqLength]
    # X[1] length_in : [seqLength]
    
    # Y[0] sequence_out : 1
    # Y[1] length_out : 1
    
    for i in range(0, inputSize, 1):
        
        sequence_in = notes[i:i + seqLength]
        sequence_out = notes[i + seqLength]
        
        length_in = noteLengths[i:i + seqLength]
        length_out = noteLengths[i + seqLength]
        
        noteIn[0].append([noteToInt[char] for char in sequence_in])
        noteIn[1].append([lengthToInt[char] for char in length_in])
        
        noteOut[0].append(noteToInt[sequence_out])
        noteOut[1].append(lengthToInt[length_out])
        
    # npatterns = len(noteIn) (inputSize)

    # reshape the input into a format compatible with LSTM layers
    # noteIn = np.reshape(noteIn, (inputSize, seqLength, 1))
    
    noteIn = np.array(noteIn, dtype=float)
    noteOut = np.array(noteOut, dtype=float)
    
    # normalize input
    noteIn[0] = noteIn[0] / float(npitch)
    noteIn[1] = noteIn[1] / float(nlengths)
    
    #print("noteIn shape:")
    #print(noteIn.shape)
    #print("noteOut shape:")
    #print(noteOut.shape)
    #print(noteIn)
    
    catNote = np_utils.to_categorical(noteOut[0])
    catLength = np_utils.to_categorical(noteOut[1])
    
    # Concatenate the input arrays row wise
    # Concatenate the categorical arrays along axis 1 (columnwise)
    
    #print("noteIn:")
    #print(noteIn)
    
    # Convert from C x N x S to N x S x C
    
    reshape = []
    for N in range(inputSize):
        for S in range(seqLength):
            for C in range(categories):
                reshape.append(noteIn[C][N][S])
    
    dout = np.concatenate((catNote, catLength), 1)
    din = np.reshape(reshape, (inputSize, seqLength, categories))
    
#     print("din shape:")
#     print(din.shape)
#     print("dout shape:")
#     print(dout.shape)
#     print("din:")
#     print(din)
    
#     print(len(catNote[0]))
#     print(len(catLength[0]))
    
#     print(din[0][0][0])
#     print(din[0][0][1])

    return (din, dout)


In [39]:
# print(np.arange(10).reshape(5,2).T.flatten())
notes, noteLengths = buildNotes()
noteIn, noteOut = prepareSeq(notes, noteLengths)
print(noteIn.shape)
print(noteOut.shape)

Parsing Music\Fugue1.mid
Parsing Music\Fugue10.mid
Parsing Music\Fugue11.mid
MIDI file Music\Fugue12.mid failed to parse
Parsing Music\Fugue13.mid
Parsing Music\Fugue14.mid
Parsing Music\Fugue15.mid
Parsing Music\Fugue16.mid
Parsing Music\Fugue17.mid
Parsing Music\Fugue18.mid
MIDI file Music\Fugue19.mid failed to parse
Parsing Music\Fugue2.mid
Parsing Music\Fugue20.mid
Parsing Music\Fugue21.mid
Parsing Music\Fugue22.mid
Parsing Music\Fugue23.mid
Parsing Music\Fugue24.mid
Parsing Music\Fugue3.mid
Parsing Music\Fugue4.mid
Parsing Music\Fugue5.mid
MIDI file Music\Fugue6.mid failed to parse
Parsing Music\Fugue7.mid
Parsing Music\Fugue8.mid
Parsing Music\Fugue9.mid
Instrument Parts 1
(22362, 16, 2)
(22362, 114)


In [52]:
def createNet(noteIn, nvocab):

#     print("In createNet()")
    
#     print(noteIn.shape)
    
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(noteIn.shape[1], noteIn.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    
    # Chris's code
    model.add(LSTM(
        512,
        input_shape=(noteIn.shape[1], noteIn.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=False
    ))
    
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation(activation='relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    
    #model.add(Flatten())
    
    model.add(Dense(nvocab))
    model.add(Lambda(lambda x: x / 0.6))
    model.add(Activation(activation='softmax'))

    model.compile(
            loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy']
        )
    
    return model

def trainNet(_epochs=1):
    """ Train a Neural Network to generate music """
    notes, noteLengths = buildNotes()

    nvocab = len(set(notes)) + len(set(noteLengths))
    
    noteIn, noteOut = prepareSeq(notes, noteLengths)
    
    model = createNet(noteIn, nvocab)
 
    checkpoint = ModelCheckpoint(
        "weights2-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5",
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )

    callbacks_list = [checkpoint]

    # Your line of code here
    model.fit(noteIn, noteOut, batch_size=1024, epochs=_epochs, verbose=1, callbacks=callbacks_list)
    


In [41]:
trainNet(100)

Parsing Music\Fugue1.mid
Parsing Music\Fugue10.mid
Parsing Music\Fugue11.mid
MIDI file Music\Fugue12.mid failed to parse
Parsing Music\Fugue13.mid
Parsing Music\Fugue14.mid
Parsing Music\Fugue15.mid
Parsing Music\Fugue16.mid
Parsing Music\Fugue17.mid
Parsing Music\Fugue18.mid
MIDI file Music\Fugue19.mid failed to parse
Parsing Music\Fugue2.mid
Parsing Music\Fugue20.mid
Parsing Music\Fugue21.mid
Parsing Music\Fugue22.mid
Parsing Music\Fugue23.mid
Parsing Music\Fugue24.mid
Parsing Music\Fugue3.mid
Parsing Music\Fugue4.mid
Parsing Music\Fugue5.mid
MIDI file Music\Fugue6.mid failed to parse
Parsing Music\Fugue7.mid
Parsing Music\Fugue8.mid
Parsing Music\Fugue9.mid
Instrument Parts 1
Epoch 1/100


KeyboardInterrupt: 

In [53]:
def generate(model):
    
    print("In Generate")
    
    notes = pickle.load(open('notes.p', 'rb'))
    noteLengths = pickle.load(open('noteLengths.p', 'rb'))

    pitchSet = sorted(set(notes))
    lengthSet = sorted(set(noteLengths))
    
    # 1. Call to prepareSeqPred
    
    noteIn = prepareSeqPred(notes, noteLengths) 
    
    # print("Model Loaded")
    
    # 2. Call to genNotes
    
    predOut = genNotes(model, noteIn, pitchSet, lengthSet)
    
    # 3. Call to createMidi
    createMidi(predOut)

# 1.
    
def prepareSeqPred(notes, noteLengths):
    
    print("In Prepare Sequences Prediction")
    
    seqLength = sequence_length_global
    inputSize = len(notes) - seqLength
    categories = 2
    
    pitchSet = sorted(set(notes))
    npitch = len(pitchSet)
    noteToInt = dict((note, number) for number, note in enumerate(pitchSet))
    
    lengthSet = sorted(set(noteLengths))
    nlengths = len(lengthSet)
    lengthToInt = dict((length, number) for number, length in enumerate(lengthSet))
    
    noteIn = [
        [],
        []
    ]
    
    noteOut = [
        [],
        []
    ]
    
    for i in range(0, inputSize, 1):
        
        sequence_in = notes[i:i + seqLength]
        length_in = noteLengths[i:i + seqLength]
        
        sequence_out = notes[i + seqLength]
        length_out = noteLengths[i + seqLength]
        
        noteIn[0].append([noteToInt[char] for char in sequence_in])
        noteOut[0].append(noteToInt[sequence_out])
        
        noteIn[1].append([lengthToInt[char] for char in length_in])
        noteOut[1].append(lengthToInt[length_out])
        
    noteIn = np.array(noteIn, dtype=float)
    noteOut = np.array(noteOut, dtype=float)
    
    # normalize input
    noteIn[0] = noteIn[0] / float(npitch)
    noteIn[1] = noteIn[1] / float(nlengths)
        
    reshape = []
    for N in range(inputSize):
        for S in range(seqLength):
            for C in range(categories):
                reshape.append(noteIn[C][N][S])
    
    din = np.reshape(reshape, (inputSize, seqLength, categories))
        
#     print("din shape:")
#     print(din.shape)
#     print("din:")
#     print(din)

    return din

# 2.

def genNotes(model, noteIn, pitchSet, lengthSet):
    
    """ Generate notes from the neural network based on a sequence of notes """
    # Starts the melody by picking a random sequence from the input as a starting point
    
    inputSize = len(noteIn)
        
    start = np.random.randint(0, inputSize - 1)
    pattern = noteIn[start] # Remember that it is an N x S x C
    
    print("pattern %s" % pattern)
        
    seqLength = len(pattern)
    categories = 2
    
    print("seqLength: %d" % seqLength)

    npitches = len(pitchSet)
    nlengths = len(lengthSet)
    
    print("npitches %d nlengths %d" % (npitches, nlengths))
    
    # TODO
    intToNote = dict((number, note) for number, note in enumerate(pitchSet))
    intToLength = dict((number, length) for number, length in enumerate(lengthSet))
    
    predOut = []
    
#     print("In genNotes(): noteIn[start] = %s" % pattern)
#     print("inputSize: %d" % inputSize)
#     print("categories: %d" % categories)
    
#     print("pitchSet %s" % pitchSet)
#     print("lengthSet %s" % lengthSet)

    for i in range(1000):
        
        # Reshape a single sample into 1 x S x C array
        
        prediction_input = np.reshape(pattern, (1, seqLength, categories))
        
#         print("prediction_input:")
#         print(prediction_input)
        
        ### Complete the line below
        prediction = model.predict(np.array(prediction_input))
#         print(len(prediction[0]))       
#         print("prediction: %s" % prediction[0])
#         print("prediction length: %d" % len(prediction[0]))
        
        predPitch = prediction[0][:npitches].astype('float64')
        predLength = prediction[0][npitches:].astype('float64')
        
#         print("Before")
#         print(predPitch)
#         print(predLength)
        
        #predPitch /= sumNote
        #predLength /= sumLength
        
#         print("sumNote %f sumLength %f" % (predPitch.sum(), predLength.sum()))
        
        predPitch /= predPitch.sum()
        predLength /= predLength.sum()
        
#         print("After")
#         print(predPitch)
#         print(predLength)
        
#         print("sumNote %f sumLength %f" % (predPitch.sum(), predLength.sum()))
        
        array = randm(1, predPitch)
#         print("array predPitch: %s" % array)
        indexPitch = np.argmax(array)
        
        array = randm(1, predLength)
#         print("array predLength: %s" % array)
        indexLength = np.argmax(array)
        
        #TODO return tuple pitch, length
        result = np.array([intToNote[indexPitch], intToLength[indexLength]])
        predOut.append(result)
        
#         print("result %s" % result)
        
        append = np.reshape(np.array([indexPitch/float(npitches), indexLength/float(nlengths)]), (1,2))
        pattern = np.concatenate((pattern, append))
        
#         print("pattern: %s" % pattern)
        
        pattern = pattern[1:len(pattern)]
        
    print(predOut)

    return predOut

# 3.
    
def createMidi(predOut):
    
    print("In createMidi()")
    
    tempo = 0.5
    
    offset = 0
    output_notes = []
    for pattern in predOut:
        if ('.' in pattern[0]) or pattern[0].isdigit():
            notes_in_chord = pattern[0].split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                if pattern[1] != '0.0':
                    new_note.quarterLength = float(pattern[1]) # Assign duration
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            new_note = note.Note(pattern[0])
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            if pattern[1] != '0.0':
                new_note.quarterLength = float(pattern[1]) # Assign duration
            output_notes.append(new_note)
        offset += tempo
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='test_output_GAN.mid')


In [ ]:
model = 0

In [ ]:
# Load a previous model (dependency)
if model == 0:
    model = load_model("weights2-improvement-50-5.9896-bigger.hdf5")
generate(model)

In [ ]:
generate(model)

In [54]:
def create_network_RNN (network_input, n_vocab):
    """ create the structure of the neural network """
    print(network_input.shape)
    model = Sequential()
    model.add(LSTM(512,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(512, return_sequences=True)))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(512)))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('relu'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

def train_network_RNN():
    """ Train a Neural Network to generate music """
    notes, noteLengths = buildNotes()

    n_vocab = len(set(notes)) + len(set(noteLengths))
    
    noteIn, noteOut = prepareSeq(notes, noteLengths)

    # Set up the model
    model = create_network_RNN(noteIn, n_vocab)
    history = History()
    
    # Fit the model
    n_epochs = 11
    model.summary()
    model.fit(noteIn, noteOut, callbacks=[history], epochs=n_epochs, batch_size=64)
    model.save('LSTMmodel.h5')   
    
    # Plot the model losses
    pd.DataFrame(history.history).plot()
    plt.savefig('LSTM_Loss_per_Epoch.png', transparent=True)
    plt.close()
    return model

In [ ]:
LSTM_model = train_network_RNN()

In [44]:
generate(LSTM_model)

In Generate
In Prepare Sequences Prediction
pattern [[0.44303797 0.02857143]
 [0.91139241 0.02857143]
 [0.56962025 0.2       ]
 [0.44303797 0.02857143]
 [0.60759494 0.02857143]
 [0.87341772 0.08571429]
 [0.55696203 0.02857143]
 [0.44303797 0.02857143]
 [0.4556962  0.2       ]
 [0.96202532 0.02857143]
 [0.62025316 0.02857143]
 [0.91139241 0.02857143]
 [0.56962025 0.02857143]
 [0.44303797 0.02857143]
 [0.4556962  0.02857143]
 [0.96202532 0.02857143]]
seqLength: 16
npitches 79 nlengths 35
[array(['B-3', '0.25'], dtype='<U4'), array(['B4', '0.0'], dtype='<U3'), array(['D4', '0.5'], dtype='<U3'), array(['F#4', '0.25'], dtype='<U4'), array(['F#4', '0.25'], dtype='<U4'), array(['B3', '0.25'], dtype='<U4'), array(['B-4', '0.25'], dtype='<U4'), array(['B-4', '0.5'], dtype='<U3'), array(['E-5', '0.75'], dtype='<U4'), array(['F#3', '0.5'], dtype='<U3'), array(['C#4', '0.25'], dtype='<U4'), array(['E-5', '0.25'], dtype='<U4'), array(['G4', '0.5'], dtype='<U3'), array(['F5', '0.25'], dtype='<U4'), 

In [75]:
class GAN():
    def __init__(self, rows, cols):
        self.seq_length = rows
        self.catagories = cols
        self.seq_shape = (rows,cols,1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        
        optimizer = tf.keras.optimizers.Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates note sequences
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(generated_seq)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_discriminator(self):

        model = Sequential()
        model.add(LSTM(512, input_shape=(self.seq_length, self.catagories), return_sequences=True))
        model.add(Bidirectional(LSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        seq = Input(shape=(self.seq_length, self.catagories))
        validity = model(seq)

        return Model(seq, validity)
      
    def build_generator(self):

        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNorm(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNorm(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNorm(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='sigmoid'))
        model.add(Reshape(self.seq_shape))
        model.summary()

        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)
        
        return Model(noise, seq)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load and convert the data
        notes, noteLengths = buildNotes()

        n_vocab = len(set(notes)) + len(set(noteLengths))
    
        X_train, y_train = prepareSeq(notes, noteLengths)

        # Adversarial ground truths
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        # Training the model
        for epoch in range(epochs):

            # Training the discriminator
            # Select a random batch of note sequences
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]

            #noise = np.random.choice(range(484), (batch_size, self.latent_dim))
            #noise = (noise-242)/242
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new note sequences
            gen_seqs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            #  Training the Generator
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as real)
            g_loss = self.combined.train_on_batch(noise, real)

            # Print the progress and save into loss lists
            if epoch % sample_interval == 0:
              print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
              self.disc_loss.append(d_loss[0])
              self.gen_loss.append(g_loss)
            
        self.plot_loss()
        
    def generate(self):
        # Get pitch names and store in a dictionary
        notes = pickle.load(open('notes.p', 'rb'))
        noteLengths = pickle.load(open('noteLengths.p', 'rb'))

        pitchSet = sorted(set(notes))
        lengthSet = sorted(set(noteLengths))
        
        npitches = len(pitchSet)
        nlengths = len(lengthSet)
        
        intToNote = dict((number, note) for number, note in enumerate(pitchSet))
        intToLength = dict((number, length) for number, length in enumerate(lengthSet))
        
        predOut = []
        for i in range(10):
            noise = np.random.normal(0, 1, (1, self.latent_dim))
            predictions = self.generator.predict(noise)
            for pattern in predictions[0]:
                indexPitch = int(pattern[0] * npitches)
                indexLength = int(pattern[1] * nlengths)
                result = np.array([intToNote[indexPitch], intToLength[indexLength]])
                predOut.append(result)
        createMidi(predOut)
        
        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('GAN_Loss_per_Epoch_final.png', transparent=True)
        plt.close()   

In [76]:
notes, noteLengths = buildNotes()
noteIn, noteOut = prepareSeq(notes, noteLengths)
shape = np.shape(noteIn)
gan = GAN(rows=shape[1], cols =shape[2])    
gan.train(epochs=500, batch_size=32, sample_interval=1)

Parsing Music\Fugue1.mid
Parsing Music\Fugue10.mid
Parsing Music\Fugue11.mid
MIDI file Music\Fugue12.mid failed to parse
Parsing Music\Fugue13.mid
Parsing Music\Fugue14.mid
Parsing Music\Fugue15.mid
Parsing Music\Fugue16.mid
Parsing Music\Fugue17.mid
Parsing Music\Fugue18.mid
MIDI file Music\Fugue19.mid failed to parse
Parsing Music\Fugue2.mid
Parsing Music\Fugue20.mid
Parsing Music\Fugue21.mid
Parsing Music\Fugue22.mid
Parsing Music\Fugue23.mid
Parsing Music\Fugue24.mid
Parsing Music\Fugue3.mid


KeyboardInterrupt: 

In [78]:
gan.generate()

In createMidi()
